# Description and Goal

This project performs cleaning, EDA, and modelling to the BlueBikes bicycle sharing system data from 20xx to 20xx found [here](https://bluebikes.com/system-data)



| Column (From website)      |
| ----------- |
| Trip Duration (seconds) |
| Start Time and Date |
| Stop Time and Date |
| Start Station Name & ID |
| End Station Name & ID |
| Bike ID |
| User Type (Casual = Single Trip or Day Pass user; Member = Annual or Monthly Member) |









# Load and Clean Data

Step 1: Preprocessing (in Python/Pandas)

Let's load data. Lots of files from the website that we need to standardize column names for and concatenate into one csv file?
- Loop through CSVs, inspect column names, standardize them.
- Concatenate all into one big DataFrame.
- Clean data (e.g., fix datetime parsing, column types, missing values).

In [63]:
import pandas as pd

In [64]:
start_path = "/Users/ellawang/Documents/GitHub/bike_csv_files/"
old_end_path = "-hubway-tripdata.csv"
new_end_path = "-bluebikes-tripdata.csv"
yr_15 = ["2015" + str(i).zfill(2) + old_end_path for i in range(1, 13)]
yr_16 = ["2016" + str(i).zfill(2) + old_end_path for i in range(1, 13)]
yr_17 = ["2017" + str(i).zfill(2) + old_end_path for i in range(1, 13)]
yr_18_1 = ["2018" + str(i).zfill(2) + old_end_path for i in range(1, 5)] 
    # note 1801-1803 i had to manually replace _ with - in the names
    # after 1805 hubway-->bluebikes
yr_18_2 = ["2018" + str(i).zfill(2) + new_end_path for i in range(5, 13)] 
yr_19 = ["2019" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_20 = ["2020" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_21 = ["2021" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_22 = ["2022" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_23 = ["2023" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_24 = ["2024" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_25 = ["2025" + str(i).zfill(2) + new_end_path for i in range(1, 7)]
pathways = yr_15 + yr_16 + yr_17 + yr_18_1 + yr_18_2 + yr_19 + yr_20 + yr_21 + yr_22 + yr_23 + yr_24 + yr_25

# condense this shii

In [67]:
# # give us a peak into the columns and formats/datatypes of each file
# num_total_rows = 0
# col_count = {}
# for path in pathways:
#     df = pd.read_csv(start_path + path)
#     print(f'{path}: {df.columns} : {df.shape[0]} rows')
#     num_total_rows += df.shape[0]
#     print(df.iloc[0])
#     for col in df.columns:
#         if col not in col_count:
#             col_count[col] = 0
#         col_count[col] += 1

# print(col_count)
# print(f'Num total rows: {num_total_rows}')

# # saved to "output.txt" so don't have to re-run

We see inconsistent naming conventions. investigated into output of the print statements printing one line from each file to see which columns are the same and of those which are reformatted and also which columns like dropped before or after a certain point

Column names in 99 files (201501 until 202303 (including final yr/mo))
- 'tripduration': 99 (ends in 202303) (e.g. 1105) -- DROPPING tentatively?
- 'bikeid': 99, (ends in 202304) (e.g. 6680) -- DROPPING tentatively?
- 'starttime': 99 (turns into started_at beg. 202304) (e.g. 2023-03-01 00:00:44.1520 --> 2023-04-13 13:49:59)
- 'stoptime': 99 (turns into ended_at beg. 202304)
- 'start station id': 99 (turns into start_station_name beg. 202304) (e.g. 386 --> A32011)
- 'start station name': 99, (turns into start_station_name beg. 202304) (e.g. Central Square at Mass Ave / Essex St --> seems to stay same!)
- start station latitude': 99, (turns into start_lat beg. 202304) (e.g. 42.368605 --> 42.363713 stays the same!)
- 'start station longitude': 99, (turns into start_lng beg. 202304) (same)
- 'end station id': 99, (turns into end_station_id beg. 202304) (e.g. 386 --> A32011 aka same)
- 'end station name': 99, (turns into end_station_name beg. 202304) (same)
- 'end station latitude': 99, (turns into end_lat beg. 202304) (same)
- 'end station longitude': 99, (turns into end_lng beg. 202304) (same)
- 'usertype': 99(turns into member_casual beg. 202304) (e.g. Customer or Subscriber --> member or casual)

Column names in 64 files (201501 until 202004)
- 'birth year': 64 (e.g. 1984) -- DROPPING
- 'gender': 64 (e.g. 0 or 1 or 2) -- DROPPING

Column names in 35 files (202005 until 202303)
- 'postal code': 35 (e.g. 02118 or NaN) -- DROPPING

Column names in 27 files (202304 to 202506)
- 'ride_id': 27 (begins 202304) (e.g. 0093AA5E7E3E0158) -- DROPPING
- 'rideable_type': 27, (begins 202304) (e.g. docked_bike or classic_bike or electric_bike) -- DROPPING tentatively?

dropping columns: I will delete birth year, gender, and postal code since those are present in only half or fewer of the rows and not the most imformative. I will drop ride_id since not informative and just distinguishes rides from each other, bikeid because I don't care too much about particular bike (not sure about htis assumption hm), dropping tripduration bc that can be deduced from starttime and endtime (i'll engineer a new col after this). 

for now, i will drop rideable type bc it's only in 27 rows... however this is a meaningful var to predict other things so will do more research (maybe bluebikes only started offering e bikes a certain year and prior to that there was only classic bike... also idk the diff between classic and docked bike lol so will look into that later but for now drop?)

In [71]:
# drop those columns - 
def load_and_clean_csv(filepath):
    
    # read_csv
    df = pd.read_csv(filepath)
    
    # rename cols as needed pass in dict
    
    # get a subset of columns wanted
    
    return df
    
    # for path in pathways:
    #     df = pd.read_csv(start_path + path)
    #     print(f'{path}: {df.columns} : {df.shape[0]} rows')
    #     num_total_rows += df.shape[0]
    #     print(df.iloc[0])
    #     for col in df.columns:
    #         if col not in col_count:
    #             col_count[col] = 0
    #         col_count[col] += 1

    # print(col_count)
    # print(f'Num total rows: {num_total_rows}')
    # return df

# get list of all pathways (this is pathways from prev code cell)

# # get list of dataframes
# dfs_list = [load_and_clean_csv(pathway) for pathway in pathways]

# # concat and get a list of that funciton applied to each pathways
# big_df = pd.concat(dfs_list, ignore_index=True)

# tEST
df_test = load_and_clean_csv(start_path + pathways[0])

df_test

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,542,2015-01-01 00:21:44,2015-01-01 00:30:47,115,Porter Square Station,42.387995,-71.119084,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075,277,Subscriber,1984,1
1,438,2015-01-01 00:27:03,2015-01-01 00:34:21,80,MIT Stata Center at Vassar St / Main St,42.361962,-71.092053,95,Cambridge St - at Columbia St / Webster Ave,42.372969,-71.094445,648,Subscriber,1985,1
2,254,2015-01-01 00:31:31,2015-01-01 00:35:46,91,One Kendall Square at Hampshire St / Portland St,42.366277,-71.091690,68,Central Square at Mass Ave / Essex St,42.365070,-71.103100,555,Subscriber,1974,1
3,432,2015-01-01 00:53:46,2015-01-01 01:00:58,115,Porter Square Station,42.387995,-71.119084,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075,1307,Subscriber,1987,1
4,735,2015-01-01 01:07:06,2015-01-01 01:19:21,105,Lower Cambridgeport at Magazine St/Riverside Rd,42.356954,-71.113687,88,Inman Square at Vellucci Plaza / Hampshire St,42.374035,-71.101427,177,Customer,1986,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7835,823,2015-01-31 22:20:49,2015-01-31 22:34:33,88,Inman Square at Vellucci Plaza / Hampshire St,42.374035,-71.101427,87,Harvard University Housing - 115 Putnam Ave at...,42.366621,-71.114214,1118,Subscriber,1992,1
7836,721,2015-01-31 22:34:32,2015-01-31 22:46:33,73,Harvard Square at Brattle St / Eliot St,42.373231,-71.120886,87,Harvard University Housing - 115 Putnam Ave at...,42.366621,-71.114214,1005,Customer,1984,1
7837,567,2015-01-31 22:36:34,2015-01-31 22:46:01,145,Rindge Avenue - O'Neill Library,42.392766,-71.129042,74,Harvard Square at Mass Ave/ Dunster,42.373268,-71.118579,854,Subscriber,1962,1
7838,197,2015-01-31 22:59:23,2015-01-31 23:02:41,70,Harvard Kennedy School at Bennett St / Eliot St,42.371196,-71.121473,97,Harvard University River Houses at DeWolfe St ...,42.369190,-71.117141,718,Subscriber,1982,1


then i'll rename columns, standardize formatting, and visualize with EDA as well as missing values before i decide how to go about filling in missing values

In [ ]:
# rename columns
# standardize formatting e.g. starttime

In [ ]:
# eda techniques

# EDA